# Combining Image Classification and NLP Models Together

## Import Libraries

In [1]:
# change the python's path to the parent directory
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import gc
import numpy as np

from models.popular_models import PopularModels
from models.lstm import LSTM
from torchvision import transforms
from tqdm import tqdm
from data.dataset import MultiLabelDataset
from tools.tools import get_data, load_data, tokenize, remove_class, count_class, calculate_pos_weights
from torch.utils.data import DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)
from sklearn.model_selection import train_test_split

/home/reynardo/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Remove unused memory in GPU

In [4]:
gc.collect()
torch.cuda.empty_cache()

## Load the Data

In [11]:
# import data
train_data = get_data("../dataset/train.csv")
test_data = get_data("../dataset/test.csv")

# perform text cleaning and get the pandas' dataframe
train_data = load_data(train_data)
test_data = load_data(test_data, has_label=False)

# remove an imbalanced class
train_data = remove_class(train_data, class_no=1)

# split into training and validating sets
X = train_data.iloc[:, 0:2]
y = train_data.iloc[:, 2:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train_data = pd.concat([X_train, y_train], axis=1)
val_data = pd.concat([X_test, y_test], axis=1)

# join the data together
for_nlp_data = pd.concat((train_data['caption'], val_data['caption'], test_data['caption']), ignore_index=True)

In [12]:
print(train_data.shape)
print(val_data.shape)
print(test_data.shape)

(12740, 21)
(3185, 21)
(10000, 2)


In [13]:
print(f"Number of training instances: 	{train_data.shape[0]}")
print(f"Number of validation instances: {val_data.shape[0]}")
print(f"Number of testing instances:  	{test_data.shape[0]}")

Number of training instances: 	12740
Number of validation instances: 3185
Number of testing instances:  	10000


## Preprocessing for Images and Caption

In [14]:
# define the image transformation: currently following resnet18
transform = transforms.Compose([
	transforms.Resize((232, 232)),
	transforms.CenterCrop(224),
	transforms.ToTensor(), # converts images to [0, 1]
	transforms.Normalize(
		mean=[0.485, 0.456, 0.406],
		std=[0.229, 0.224, 0.225],
	)
])

# tokenize the data
final_list, vocab = tokenize(for_nlp_data)
X_train_vec = final_list[:train_data.shape[0], :]
X_val_vec = final_list[train_data.shape[0]:train_data.shape[0]+val_data.shape[0], :]
X_test_vec = final_list[train_data.shape[0]+val_data.shape[0]:, :]

print(final_list.shape)
print(X_train_vec.shape)
print(X_val_vec.shape)
print(X_test_vec.shape)

Max Sentence Length: 28
(25925, 28)
(12740, 28)
(3185, 28)
(10000, 28)


## Create Dataset and DataLoader

In [15]:
# initialize the dataset
train_dataset = MultiLabelDataset(
	csv_file=train_data,
	root_dir='../dataset/data/',
	vectorizer=None,
	transform=transform,
	use_caption_vec=True,
	caption_vec=X_train_vec,
	is_test=False,
)
val_dataset = MultiLabelDataset(
    csv_file=val_data,
    root_dir='../dataset/data/',
    vectorizer=None,
	transform=transform,
	use_caption_vec=True,
	caption_vec=X_val_vec,
	is_test=False,
)
test_dataset = MultiLabelDataset(
	csv_file=test_data,
	root_dir='../dataset/data/',
	vectorizer=None,
	transform=transform,
	use_caption_vec=True,
	caption_vec=X_test_vec,
	is_test=True,
)

BATCH_SIZE=16

# load the dataset into batches 
train_dataloader = DataLoader(
	dataset=train_dataset,
	batch_size=BATCH_SIZE,
	shuffle=True,
)
val_dataloader = DataLoader(
    dataset=val_dataset,
	batch_size=BATCH_SIZE,
	shuffle=True,
)
test_dataloader = DataLoader(
	dataset=test_dataset,
	batch_size=BATCH_SIZE,
	shuffle=False,
)

## Create the Combined Model

In [16]:
class CombinedModel(nn.Module):

	def __init__(
			self, 
			choice: str, 
			pretrained: bool, 
			freeze: bool, 
			cnn_n_out: int,
			no_layers: int,
			vocab_size: int,
			embedding_dim: int,
			lstm_hidden_dim: int,
			lstm_n_out: int,
			lstm_dropout: float = 0.5,
			fc_dropout: float = 0.5,
		) -> None:
		super(CombinedModel, self).__init__()

		# get the CNN model for image classification
		self.cnn_model = PopularModels(
			choice=choice,
			pretrained=pretrained,
			freeze=freeze,
			n_out=cnn_n_out,
		).get_model()

		# get the LSTM model for text classification
		self.lstm_model = LSTM(
			no_layers=no_layers,
			vocab_size=vocab_size + 1,
			embedding_dim=embedding_dim,
			hidden_dim=lstm_hidden_dim,
			output_dim=lstm_n_out,
			dropout=lstm_dropout,
		)
		
		# create the final fully connected layer
		self.last_layer = nn.Linear(cnn_n_out + lstm_n_out, 19)
		
		# prevent overfitting
		self.dropout_layer = nn.Dropout(p=fc_dropout, inplace=True)

	def forward(self, x, y, hidden):
		x = self.cnn_model(x)
		y, hidden = self.lstm_model(y, hidden)
		
		output = torch.cat((x, y), dim=1)
		output = self.dropout_layer(output)
		output = self.last_layer(output)

		return output, hidden

## Define the model

In [17]:
class_counts = list(count_class(train_data).values())
pos_weights = calculate_pos_weights(class_counts, train_data)
if torch.cuda.is_available():
	pos_weights = pos_weights.to('cuda')

In [18]:
EPOCHS = 50
THRESHOLD = 0.5
N_CLASSES = 19

model = CombinedModel(
	choice="regnet_x_1_6gf",
	pretrained=True, 		# if pretrained is False, then freeze should also be False
	freeze=True,
	cnn_n_out=256,
	no_layers=2,
	vocab_size=len(vocab), 	# already added by 1
	embedding_dim=64,
	lstm_hidden_dim=256,
	lstm_n_out=128,
	lstm_dropout=0.3,
	fc_dropout=0.5,
)
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
optimizer = torch.optim.Adam(
	params=model.parameters(),
	lr=0.001,
)

# utilise GPU
if torch.cuda.is_available():
	print('using GPU')
	model = model.to('cuda')

using GPU


## Train the combined model

In [19]:
clip = 5 # for exploding gradient problem
train_losses = []
train_accs = []
sk_val_accs = []
sk_train_accs = []
for epoch in range(EPOCHS):

	# initialize the hidden state
	hidden = model.lstm_model.init_hidden(batch_size=BATCH_SIZE)

	start = 0 # index
	y_true = np.zeros((train_data.shape[0], N_CLASSES))
	y_preds = np.zeros((train_data.shape[0], N_CLASSES))
	n_total = 0
	n_correct = 0
	train_loss = 0.
	model.train()
	for image_names, images, captions, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training: "):

		if torch.cuda.is_available():
			images = images.to('cuda')
			captions = captions.to('cuda')
			labels = labels.to('cuda')
		
		# creating new variables for the hidden state, otherwise
		# we'd backprop through the entire training history
		hidden = tuple([each.data for each in hidden])

		optimizer.zero_grad()
		y_pred, hidden = model(images, captions, hidden)

		# backward
		loss = loss_fn(y_pred, labels)
		loss.backward()

		# clip_grad_norm helps prevent the exploding gradient problem in RNNs/LSTMs
		nn.utils.clip_grad_norm_(model.parameters(), clip)

		# update
		optimizer.step()

		# compare
		predicted = (F.sigmoid(y_pred) > THRESHOLD).int()

		n = images.shape[0]
		y_true[start:start+n] = labels.cpu().numpy()
		y_preds[start:start+n] = predicted.cpu().numpy()
		start = start + n

		train_loss += loss.item()
		n_correct += torch.all(torch.eq(predicted, labels), dim=1).sum()
		n_total += labels.shape[0]

	train_losses.append(train_loss / len(train_dataloader))
	train_accs.append(n_correct / n_total)
	sk_train_accs.append(accuracy_score(y_true, y_preds))

	print("Epoch {:d}, Train Loss: {:.9f}, Train Accuracy: {:.7f}%".format(epoch+1, 
																		train_losses[-1], 
																		accuracy_score(y_true, y_preds)*100), end="")
	
	# initialize the hidden state
	val_hidden = model.lstm_model.init_hidden(batch_size=BATCH_SIZE)

	start = 0 # index
	y_true = np.zeros((val_data.shape[0], N_CLASSES))
	y_preds = np.zeros((val_data.shape[0], N_CLASSES))
	model.eval()
	for (_, images, captions, labels) in val_dataloader:

		# creating new variables for the hidden state, otherwise
		# we'd backprop through the entire training history
		val_hidden = tuple([each.data for each in val_hidden])
		
		if torch.cuda.is_available():
			images = images.to('cuda')
			captions = captions.to('cuda')
			labels = labels.to('cuda')
		
		outputs, _ = model(images, captions, val_hidden)
		predicted = (F.sigmoid(outputs) > THRESHOLD).int()

		n = images.shape[0]
		y_true[start:start+n] = labels.cpu().numpy()
		y_preds[start:start+n] = predicted.cpu().numpy()
		start = start + n
	
	sk_val_accs.append(accuracy_score(y_true, y_preds))
	print("\tVal Accuracy: {:.7f}%".format(accuracy_score(y_true, y_preds)*100))

Epoch 1 Training: 100%|██████████| 797/797 [00:52<00:00, 15.13it/s]


Epoch 1, Train Loss: 0.718787902, Train Accuracy: 5.6279435%	Val Accuracy: 14.9136578%


Epoch 2 Training: 100%|██████████| 797/797 [00:52<00:00, 15.15it/s]


Epoch 2, Train Loss: 0.582069074, Train Accuracy: 12.5117739%	Val Accuracy: 20.6279435%


Epoch 3 Training: 100%|██████████| 797/797 [00:52<00:00, 15.25it/s]


...

Epoch 48, Train Loss: 0.074698429, Train Accuracy: 68.0690738%	Val Accuracy: 34.9136578%


Epoch 49 Training: 100%|██████████| 797/797 [00:52<00:00, 15.24it/s]


Epoch 49, Train Loss: 0.068741509, Train Accuracy: 69.0894819%	Val Accuracy: 36.5463108%


Epoch 50 Training: 100%|██████████| 797/797 [00:52<00:00, 15.23it/s]


Epoch 50, Train Loss: 0.074096818, Train Accuracy: 68.8932496%	Val Accuracy: 34.5682889%


In [20]:
print(sk_val_accs)
print(sk_train_accs)
print(train_losses)

[0.14913657770800628, 0.20627943485086342, 0.22762951334379905, 0.22888540031397175, 0.2345368916797488, 0.25934065934065936, 0.2759811616954474, 0.2753532182103611, 0.3189952904238619, 0.29073783359497646, 0.26907378335949766, 0.3130298273155416, 0.3058084772370487, 0.2879120879120879, 0.31993720565149136, 0.3007849293563579, 0.30800627943485087, 0.3205651491365777, 0.33312401883830456, 0.3293563579277865, 0.34631083202511775, 0.341287284144427, 0.3532182103610675, 0.3544740973312402, 0.33877551020408164, 0.32150706436420723, 0.3500784929356358, 0.33971742543171113, 0.33092621664050237, 0.3510204081632653, 0.33249607535321823, 0.341287284144427, 0.3579277864992151, 0.34945054945054943, 0.3579277864992151, 0.35133437990580846, 0.36106750392464676, 0.3315541601255887, 0.35886970172684457, 0.3469387755102041, 0.34097331240188383, 0.3456828885400314, 0.3572998430141287, 0.3576138147566719, 0.36138147566719, 0.33940345368916797, 0.3554160125588697, 0.34913657770800627, 0.3654631083202512, 

: 

## Trained standalone CNN models

In [14]:
models = [
    "mobilenet_v3_large",
	"resnet18",
	"efficientnet_b2",
	"densenet121",
	"mnasnet1_3",
	"regnet_x_1_6gf",
]
val_accs = {}
train_accs = {}
losses = {}

EPOCHS=20
THRESHOLD = 0.5
N_CLASSES = 19

# train each model
for model_name in models:

	print("Training using", model_name)

	# define model
	model = PopularModels(
		choice=model_name,
		pretrained=True,
		freeze=True,
		n_out=N_CLASSES,
	).get_model()

	# define loss function and optimizer
	loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
	optimizer = torch.optim.Adam(
		params=model.parameters(),
		lr=0.001,
	)

	# utilise GPU
	if torch.cuda.is_available():
		print('using GPU')
		model = model.to('cuda')

	# train the model
	train_losses = []
	sk_val_accs = []
	sk_train_accs = []
	for epoch in range(EPOCHS):

		start = 0 # index
		y_true = np.zeros((train_data.shape[0], N_CLASSES))
		y_preds = np.zeros((train_data.shape[0], N_CLASSES))
		train_loss = 0.
		model.train()
		for image_names, images, _, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training: "):

			if torch.cuda.is_available():
				images = images.to('cuda')
				labels = labels.to('cuda')

			optimizer.zero_grad()
			y_pred = model(images)

			# backward
			loss = loss_fn(y_pred, labels)
			loss.backward()

			# update
			optimizer.step()

			# compare
			predicted = (F.sigmoid(y_pred) > THRESHOLD).int()

			n = images.shape[0]
			y_true[start:start+n] = labels.cpu().numpy()
			y_preds[start:start+n] = predicted.cpu().numpy()
			start = start + n

			train_loss += loss.item()

		train_losses.append(train_loss / len(train_dataloader))
		sk_train_accs.append(accuracy_score(y_true, y_preds))
		print("Epoch {:d}, Train Loss: {:.9f}, Train Accuracy: {:.7f}%".format(epoch+1, 
																			train_losses[-1], 
																			accuracy_score(y_true, y_preds)*100), end="")

		start = 0 # index
		y_true = np.zeros((val_data.shape[0], N_CLASSES))
		y_preds = np.zeros((val_data.shape[0], N_CLASSES))
		model.eval()
		for (_, images, _, labels) in val_dataloader:
			
			if torch.cuda.is_available():
				images = images.to('cuda')
				labels = labels.to('cuda')
			
			outputs = model(images)
			predicted = (F.sigmoid(outputs) > THRESHOLD).int()

			n = images.shape[0]
			y_true[start:start+n] = labels.cpu().numpy()
			y_preds[start:start+n] = predicted.cpu().numpy()
			start = start + n
		
		sk_val_accs.append(accuracy_score(y_true, y_preds))
		print("\tVal Accuracy: {:.7f}%".format(accuracy_score(y_true, y_preds)*100))

	# add the accs and losses
	train_accs[model_name] = sk_train_accs
	val_accs[model_name] = sk_val_accs
	losses[model_name] = train_losses

Training using mobilenet_v3_large
using GPU


Epoch 1 Training: 100%|██████████| 797/797 [00:33<00:00, 24.13it/s]


Epoch 1, Train Loss: 0.746159702, Train Accuracy: 3.6577708%	Val Accuracy: 8.6656201%


Epoch 2 Training: 100%|██████████| 797/797 [00:33<00:00, 24.14it/s]


Epoch 2, Train Loss: 0.632567421, Train Accuracy: 7.8492936%	Val Accuracy: 9.5447410%


...

Epoch 18, Train Loss: 0.473803224, Train Accuracy: 15.6671900%	Val Accuracy: 16.2323391%


Epoch 19 Training: 100%|██████████| 797/797 [00:47<00:00, 16.73it/s]


Epoch 19, Train Loss: 0.473433698, Train Accuracy: 15.8398744%	Val Accuracy: 15.0078493%


Epoch 20 Training: 100%|██████████| 797/797 [00:48<00:00, 16.57it/s]


Epoch 20, Train Loss: 0.474110210, Train Accuracy: 15.9262166%	Val Accuracy: 16.8916797%


In [20]:
pd.DataFrame(train_accs).to_csv("../logs/cnn_only_models_train_accuracies", index=False)
pd.DataFrame(val_accs).to_csv("../logs/cnn_only_models_val_accuracies", index=False)
pd.DataFrame(losses).to_csv("../logs/cnn_only_models_losses", index=False)

: 

## Graphs and Evaluation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.title("Training Losses vs Epoch")
plt.plot(train_losses)
plt.ylabel("Epoch")
plt.xlabel("Loss")
plt.show()

In [ ]:
new_train_accs = []
for train_acc in train_accs:
    new_train_accs.append(train_acc.item())

plt.title("Training Accuracies vs Epoch")
plt.plot(new_train_accs)
plt.ylabel("Accuracy")
plt.xlabel("Loss")
plt.show()

## Save, Test and Load Model

In [ ]:
torch.save(model.state_dict(), '../combined_model.pth')

In [ ]:
model = CombinedModel(
	choice="regnet_x_1_6gf",
	pretrained=True, 		# if pretrained is False, then freeze should also be False
	freeze=True,
	cnn_n_out=256,
	no_layers=2,
	vocab_size=len(vocab), 	# already added by 1
	embedding_dim=64,
	lstm_hidden_dim=256,
	lstm_n_out=128,
)
model.load_state_dict(torch.load('../combined_model.pth'))

In [ ]:
# create a file for test submission
f = open('../submission_combined.csv', "w")
f.write("ImageID,Labels\n")

# utilise GPU
if torch.cuda.is_available():
	print('using GPU')
	model = model.to('cuda')

# initialize the hidden state
hidden = model.lstm_model.init_hidden(batch_size=BATCH_SIZE)

# idx = 0
model.eval()
for (image_names, images, captions) in test_dataloader:

	# creating new variables for the hidden state, otherwise
	# we'd backprop through the entire training history
	hidden = tuple([each.data for each in hidden])
	
	if torch.cuda.is_available():
		images = images.to('cuda')
		captions = captions.to('cuda')
	
	outputs, _ = model(images, captions, hidden)
	predicted = (F.sigmoid(outputs) > THRESHOLD).int()

	# NOTE: add 1 to the output of predicted!
	# write the output
	for i, predicted_label in enumerate(predicted):
		label = (predicted_label == torch.max(predicted_label)).nonzero().flatten()
		label += 1
		label = label.tolist()
		label = " ".join(str(x) for x in label)

		f.write(image_names[i].split("/")[-1] + "," + str(label) +"\n")

f.close()

In [ ]:
# create a file for test submission
f = open('../train_combined.csv', "w")
f.write("ImageID,Labels\n")

# utilise GPU
if torch.cuda.is_available():
	print('using GPU')
	model = model.to('cuda')

# initialize the hidden state
hidden = model.lstm_model.init_hidden(batch_size=BATCH_SIZE)

# idx = 0
model.eval()
for (image_names, images, captions, _) in train_dataloader:

	# creating new variables for the hidden state, otherwise
	# we'd backprop through the entire training history
	hidden = tuple([each.data for each in hidden])
	
	if torch.cuda.is_available():
		images = images.to('cuda')
		captions = captions.to('cuda')
	
	outputs, _ = model(images, captions, hidden)
	predicted = (F.sigmoid(outputs) > THRESHOLD).int()

	# NOTE: add 1 to the output of predicted!
	# write the output
	for i, predicted_label in enumerate(predicted):
		label = (predicted_label == torch.max(predicted_label)).nonzero().flatten()
		label += 1
		label = label.tolist()
		label = " ".join(str(x) for x in label)

		f.write(image_names[i].split("/")[-1] + "," + str(label) +"\n")

f.close()